In [96]:
import os
import math
import numpy as np

import tensorflow as tf

import glob
import matplotlib.pyplot as plt
#pre trained inception model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.optimizers import SGD
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
tf.compat.v1.disable_eager_execution()

#image generator
from keras.preprocessing.image import ImageDataGenerator

print("Tensorflow version " + tf.__version__)

Tensorflow version 1.14.0


In [77]:
#Count number of files in folder and subfolders
def get_nb_files (path):
    if not os.path.exists(path):
        return 0
    return sum([len(files) for r, d, files in os.walk(path)])

In [78]:
#Count the number of subfolders, which in this case are the clases.
def get_nb_subfolders(path):
    if not os.path.exists(path):
        return 0
    return sum([len(d) for r, d, files in os.walk(path)])

In [79]:
#Define image generators:
def create_img_generator():
    return ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [87]:
#Main
Image_width, Image_height = 299, 299
Training_Epochs = 2
Batch_size = 32 #number of images in batch
Number_FC_Neurons = 1024

train_dir = './data/train'
validate_dir = './data/val' 

num_trian_samples = get_nb_files(train_dir)
num_classes = get_nb_subfolders(train_dir)
num_validate_samples = get_nb_files(validate_dir)

In [88]:
### Define data preprocessing ###

#Define image generators for training and testing
train_image_gen = create_img_generator()
test_image_gen = create_img_generator()

#Training image generator
train_generator = train_image_gen.flow_from_directory(
    train_dir, target_size=(Image_width,Image_height), batch_size=Batch_size,
    seed = 42)

#Validation image generator(Define the generator for the validation folder)
validation_generator = test_image_gen.flow_from_directory(
    validate_dir,
    target_size=(Image_width,Image_height),
    batch_size=Batch_size,
    seed=42)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [97]:
#Load the inception v3 model
InceptionV3_base_model = InceptionV3 (weights = 'imagenet', include_top = False)
print('Inception V3 base model without last FC loaded')






87916544/87910968 [==============================] - ETA: 10 - ETA: 5: - ETA: 5: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 58s - ETA: 58 - ETA: 56 - ETA: 54 - ETA: 54 - ETA: 53 - ETA: 51 - ETA: 51 - ETA: 49 - ETA: 48 - ETA: 48 - ETA: 47 - ETA: 46 - ETA: 46 - ETA: 45 - ETA: 44 - ETA: 43 - ETA: 43 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 40 - ETA: 40 - ETA: 39 - ETA: 39 - ETA: 39 - ETA: 38 - ETA: 38 - ETA: 39 - ETA: 41 - ETA: 42 - ETA: 43 - ETA: 42 - ETA: 44 - ETA: 41 - ETA: 40 - ETA: 41 - ETA: 40 - ETA: 39 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 39 - ETA: 38 - ETA: 38 - ETA: 38 - ETA: 37 - ETA: 37 - ETA: 37 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 35 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 34 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - ETA: 33 - E

In [98]:
#define the layers in the nes classification prediction
x = InceptionV3_base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(Number_FC_Neurons, activation='relu')(x) #new FC layer
predictions = Dense(num_classes,activation = 'softmax')(x) #new softmax layer


In [99]:
#Define new model containing the first layers from de inception v3 base and the new classifier
model = Model(inputs = InceptionV3_base_model.input, outputs = predictions) 

In [100]:
#Print model structure diagram
print (model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 3, None, None 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, None, Non 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, None, Non 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, None, Non 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

__________________________________________________________________________________________________
mixed0 (Concatenate)            (None, 256, None, No 0           activation_6[0][0]               
                                                                 activation_8[0][0]               
                                                                 activation_11[0][0]              
                                                                 activation_12[0][0]              
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 64, None, Non 16384       mixed0[0][0]                     
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 64, None, Non 192         conv2d_16[0][0]                  
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
batch_normalization_20 (BatchNo (None, 64, None, Non 192         conv2d_20[0][0]                  
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 64, None, Non 192         conv2d_22[0][0]                  
__________________________________________________________________________________________________
batch_normalization_25 (BatchNo (None, 96, None, Non 288         conv2d_25[0][0]                  
__________________________________________________________________________________________________
batch_normalization_26 (BatchNo (None, 64, None, Non 192         conv2d_26[0][0]                  
__________________________________________________________________________________________________
activation_20 (Activation)      (None, 64, None, Non 0           batch_normalization_20[0][0]     
__________

batch_normalization_38 (BatchNo (None, 128, None, No 384         conv2d_38[0][0]                  
__________________________________________________________________________________________________
activation_33 (Activation)      (None, 128, None, No 0           batch_normalization_33[0][0]     
__________________________________________________________________________________________________
activation_38 (Activation)      (None, 128, None, No 0           batch_normalization_38[0][0]     
__________________________________________________________________________________________________
average_pooling2d_4 (AveragePoo (None, 768, None, No 0           mixed3[0][0]                     
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 192, None, No 147456      mixed3[0][0]                     
__________________________________________________________________________________________________
conv2d_34 

__________________________________________________________________________________________________
batch_normalization_41 (BatchNo (None, 192, None, No 576         conv2d_41[0][0]                  
__________________________________________________________________________________________________
batch_normalization_44 (BatchNo (None, 192, None, No 576         conv2d_44[0][0]                  
__________________________________________________________________________________________________
batch_normalization_49 (BatchNo (None, 192, None, No 576         conv2d_49[0][0]                  
__________________________________________________________________________________________________
batch_normalization_50 (BatchNo (None, 192, None, No 576         conv2d_50[0][0]                  
__________________________________________________________________________________________________
activation_41 (Activation)      (None, 192, None, No 0           batch_normalization_41[0][0]     
__________

activation_60 (Activation)      (None, 192, None, No 0           batch_normalization_60[0][0]     
__________________________________________________________________________________________________
mixed6 (Concatenate)            (None, 768, None, No 0           activation_51[0][0]              
                                                                 activation_54[0][0]              
                                                                 activation_59[0][0]              
                                                                 activation_60[0][0]              
__________________________________________________________________________________________________
conv2d_65 (Conv2D)              (None, 192, None, No 147456      mixed6[0][0]                     
__________________________________________________________________________________________________
batch_normalization_65 (BatchNo (None, 192, None, No 576         conv2d_65[0][0]                  
__________

batch_normalization_74 (BatchNo (None, 192, None, No 576         conv2d_74[0][0]                  
__________________________________________________________________________________________________
activation_74 (Activation)      (None, 192, None, No 0           batch_normalization_74[0][0]     
__________________________________________________________________________________________________
conv2d_71 (Conv2D)              (None, 192, None, No 147456      mixed7[0][0]                     
__________________________________________________________________________________________________
conv2d_75 (Conv2D)              (None, 192, None, No 258048      activation_74[0][0]              
__________________________________________________________________________________________________
batch_normalization_71 (BatchNo (None, 192, None, No 576         conv2d_71[0][0]                  
__________________________________________________________________________________________________
batch_norm

activation_84 (Activation)      (None, 384, None, No 0           batch_normalization_84[0][0]     
__________________________________________________________________________________________________
batch_normalization_85 (BatchNo (None, 192, None, No 576         conv2d_85[0][0]                  
__________________________________________________________________________________________________
activation_77 (Activation)      (None, 320, None, No 0           batch_normalization_77[0][0]     
__________________________________________________________________________________________________
mixed9_0 (Concatenate)          (None, 768, None, No 0           activation_79[0][0]              
                                                                 activation_80[0][0]              
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 768, None, No 0           activation_83[0][0]              
          

                                                                 mixed9_1[0][0]                   
                                                                 concatenate_2[0][0]              
                                                                 activation_94[0][0]              
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 2048)         0           mixed10[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         2098176     global_average_pooling2d_1[0][0] 
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 2)            2050        dense_1[0][0]                    
Total params: 23,903,010
Trainable params: 23,868,578
Non-trainable params: 34,432
__________________________

In [105]:
#transfer learning
for layer in InceptionV3_base_model.layers:
    layer.trainable = False
    
#Define model compile for transfer learning
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [107]:
#Fit the transfer learning model
history_transfer_learning = model.fit_generator(
    train_generator, #training data
    epochs=NB_EPOCHS,
    steps_per_epoch =num_trian_samples,
    validation_data=validation_generator,
    validation_steps = num_validate_samples,
    class_weight ='auto')

Epoch 1/3


 185/5219 [>.............................] - ETA: 26:03:11 - loss: 0.3930 - acc: 0.750 - ETA: 24:46:13 - loss: 0.4057 - acc: 0.781 - ETA: 24:56:11 - loss: 0.4234 - acc: 0.802 - ETA: 25:45:35 - loss: 0.4166 - acc: 0.828 - ETA: 25:46:42 - loss: 0.4013 - acc: 0.843 - ETA: 25:32:46 - loss: 0.4029 - acc: 0.822 - ETA: 25:24:54 - loss: 0.3993 - acc: 0.821 - ETA: 25:17:06 - loss: 0.4055 - acc: 0.820 - ETA: 25:18:03 - loss: 0.3922 - acc: 0.822 - ETA: 25:14:43 - loss: 0.3908 - acc: 0.821 - ETA: 25:25:28 - loss: 0.3911 - acc: 0.818 - ETA: 25:40:41 - loss: 0.3886 - acc: 0.817 - ETA: 25:35:27 - loss: 0.3897 - acc: 0.817 - ETA: 25:31:06 - loss: 0.3895 - acc: 0.812 - ETA: 25:27:16 - loss: 0.3909 - acc: 0.812 - ETA: 25:23:51 - loss: 0.3856 - acc: 0.816 - ETA: 25:19:53 - loss: 0.3795 - acc: 0.823 - ETA: 25:22:04 - loss: 0.3869 - acc: 0.810 - ETA: 25:19:50 - loss: 0.3806 - acc: 0.815 - ETA: 25:25:22 - loss: 0.3731 - acc: 0.821 - ETA: 25:22:44 - loss: 0.3703 - acc: 0.825 - ETA: 25:21:21 - loss: 0.3663 - 

 371/5219 [=>............................] - ETA: 19:47:21 - loss: 0.2616 - acc: 0.888 - ETA: 19:46:51 - loss: 0.2615 - acc: 0.888 - ETA: 19:45:37 - loss: 0.2609 - acc: 0.888 - ETA: 19:44:24 - loss: 0.2606 - acc: 0.888 - ETA: 19:43:12 - loss: 0.2608 - acc: 0.888 - ETA: 19:41:57 - loss: 0.2608 - acc: 0.888 - ETA: 19:40:45 - loss: 0.2609 - acc: 0.888 - ETA: 19:39:34 - loss: 0.2607 - acc: 0.888 - ETA: 19:38:23 - loss: 0.2615 - acc: 0.888 - ETA: 19:37:13 - loss: 0.2609 - acc: 0.888 - ETA: 19:36:02 - loss: 0.2612 - acc: 0.888 - ETA: 19:34:57 - loss: 0.2614 - acc: 0.888 - ETA: 19:33:52 - loss: 0.2610 - acc: 0.888 - ETA: 19:32:50 - loss: 0.2603 - acc: 0.889 - ETA: 19:31:49 - loss: 0.2605 - acc: 0.888 - ETA: 19:30:43 - loss: 0.2605 - acc: 0.888 - ETA: 19:29:38 - loss: 0.2597 - acc: 0.889 - ETA: 19:28:34 - loss: 0.2600 - acc: 0.888 - ETA: 19:28:00 - loss: 0.2591 - acc: 0.889 - ETA: 19:27:23 - loss: 0.2589 - acc: 0.889 - ETA: 19:26:19 - loss: 0.2587 - acc: 0.889 - ETA: 19:25:22 - loss: 0.2579 - 

 459/5219 [=>............................] - ETA: 68:45:44 - loss: 0.2448 - acc: 0.895 - ETA: 68:37:03 - loss: 0.2447 - acc: 0.895 - ETA: 68:28:43 - loss: 0.2442 - acc: 0.896 - ETA: 68:20:42 - loss: 0.2440 - acc: 0.896 - ETA: 68:12:17 - loss: 0.2435 - acc: 0.896 - ETA: 68:03:40 - loss: 0.2431 - acc: 0.896 - ETA: 67:55:05 - loss: 0.2431 - acc: 0.896 - ETA: 67:46:37 - loss: 0.2426 - acc: 0.896 - ETA: 67:38:17 - loss: 0.2425 - acc: 0.897 - ETA: 67:30:00 - loss: 0.2423 - acc: 0.897 - ETA: 67:21:59 - loss: 0.2422 - acc: 0.897 - ETA: 67:13:46 - loss: 0.2425 - acc: 0.896 - ETA: 67:05:32 - loss: 0.2428 - acc: 0.896 - ETA: 66:57:16 - loss: 0.2426 - acc: 0.897 - ETA: 66:49:02 - loss: 0.2424 - acc: 0.897 - ETA: 66:40:53 - loss: 0.2424 - acc: 0.897 - ETA: 66:32:46 - loss: 0.2420 - acc: 0.897 - ETA: 66:24:39 - loss: 0.2418 - acc: 0.897 - ETA: 66:16:45 - loss: 0.2417 - acc: 0.897 - ETA: 66:08:53 - loss: 0.2417 - acc: 0.897 - ETA: 66:01:01 - loss: 0.2416 - acc: 0.897 - ETA: 65:53:25 - loss: 0.2413 - 

KeyboardInterrupt: 